In [1]:
import pandas as pd

movies = pd.read_csv("data/ratings_movies.csv")

In [3]:
#библиотека для регулярных выражений
import re 

def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  100836 non-null  int64  
 1   userId      100836 non-null  int64  
 2   movieId     100836 non-null  int64  
 3   rating      100836 non-null  float64
 4   date        100836 non-null  object 
 5   title       100836 non-null  object 
 6   genres      100836 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 5.4+ MB


In [10]:
movies["year release"] = movies["title"].apply(get_year_release)

In [13]:
movies["year release"].isnull().sum()

18

In [42]:
mask = (movies["year release"] == 1999.0)
movies[mask].groupby(by="title")["rating"].mean().sort_values()

title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [39]:
mask = (movies["year release"] == 2010.0)
movies[mask].groupby(by="genres")["rating"].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [45]:
movies.groupby(by="userId")["genres"].nunique().sort_values()

userId
214     13
85      13
245     13
494     15
578     15
      ... 
474    395
380    399
448    403
414    482
599    524
Name: genres, Length: 610, dtype: int64

In [52]:
movies.groupby(by="userId")["rating"].agg(["mean", "count"]).sort_values(by=["count", "mean"], ascending=[True, False])

,mean,count
userId,,
53,5.000000,20
595,4.200000,20
189,4.100000,20
569,4.000000,20
278,3.875000,20
...,...,...
274,3.235884,1346
448,2.847371,1864
474,3.398956,2108


In [60]:
mask1 = (movies["year release"] == 2018.0)
movies_2018 = movies[mask1].groupby(by="genres")["rating"].agg(["mean","count"])
movies_2018[movies_2018["count"] > 10]

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [61]:
movies["year_rating"] = pd.to_datetime(movies["date"]).dt.year

In [75]:
pivot_table_movies = movies.pivot_table(
    values="rating",
    index="year_rating",
    columns="genres",
    aggfunc="mean",
    fill_value=0
)
pivot_table_movies["Comedy"]

year_rating
1996    3.228571
1997    3.409091
1998    3.000000
1999    3.606061
2000    3.141291
2001    3.318408
2002    3.198556
2003    3.120066
2004    3.356877
2005    2.963325
2006    3.347534
2007    2.928187
2008    3.426667
2009    3.273292
2010    3.179825
2011    3.232877
2012    3.478477
2013    3.489474
2014    3.271429
2015    3.106183
2016    3.373431
2017    2.852668
2018    3.121296
Name: Comedy, dtype: float64